In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import os
import pandas as pd
from nltk.stem import WordNetLemmatizer
import math
from nltk.corpus import stopwords

In [ ]:


lemmatizer = WordNetLemmatizer()
stpwrds = set(stopwords.words('english'))
docs = {}  #<fileid : filename>
terms = {}    #<word : {docid:tf-idf}>
count_of_terms_in_doc = {} #<docid:count>   total number of words in each doc
term_in_docs = {} #<term:set containing docids>      docs a term appears in

number_of_documents = 0    #each row is a doc

i = 1
for f in os.listdir("/gdrive/My Drive/AIR/TelevisionNews/"):
	docs[str(i)] = f
	i += 1
for f_id in docs:
  try:
    df = pd.read_csv("/gdrive/My Drive/AIR/TelevisionNews/"+docs[f_id])
    data = df["Snippet"]
  except:
    print(docs[f_id])
    continue

  for row in range(len(data)):
    number_of_documents += 1
    nwords = 0
    string = data[row].replace(',',' ')
    string = string.replace('.', ' ')
    string = string.replace('?', ' ')
    string = string.replace('!', ' ')
    string = string.replace('(', ' ')
    string = string.replace(')', ' ')
    string = string.replace('[', ' ')
    string = string.replace(']', ' ')
    string = string.replace(':', ' ')
    string = string.replace(';', ' ')
    string = string.replace('"', ' ')
    string = string.replace('-', ' ')
    string = string.replace('/', ' ')
    for word in string.split():
      if word in stpwrds:
        continue
      nwords += 1
      lw = lemmatizer.lemmatize(word.lower())
      if lw not in terms:
        terms[lw]={}
      if (f_id+"_"+str(row+1)) not in terms[lw]:
        terms[lw][f_id+"_"+str(row+1)]=0
        # print(terms)
      terms[lw][f_id+"_"+str(row+1)]+=1
      
      if lw not in term_in_docs:
        term_in_docs[lw] = set()
      term_in_docs[lw].add(f_id)
    
    count_of_terms_in_doc[f_id+"_"+str(row+1)] = nwords

print(len(terms))

CNN.200910.csv
41398


In [ ]:
print(number_of_documents)

94858


In [ ]:
for term in terms:
  for docid in terms[term]:
    tf = (terms[term][docid]/count_of_terms_in_doc[docid])
    idf = math.log(number_of_documents/len(term_in_docs[term]))
    terms[term][docid] = (tf,idf)

#docid is a combination of file id and row number seperated by _(underscore)
#Finally terms is a dictionary with word as a key and value is another dictionary which has doc_id as key and tf-idf as its value


In [ ]:
import json 
with open("/gdrive/My Drive/AIR/output.json", "w") as outfile: 
	json.dump(terms, outfile) 


In [ ]:
# count=0
# for term in terms:
#   if not term.isalpha(): 
#     count+=1
#     print(term)
# print(count)